In [536]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


#### Загрузка даных

In [537]:

# Get the same results each time
np.random.seed(0)


# Load the training data
data = pd.read_csv("data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

#### Исследование и обработка данных

In [538]:
comments.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: comment_text
Non-Null Count  Dtype 
--------------  ----- 
90902 non-null  object
dtypes: object(1)
memory usage: 710.3+ KB


В датасете отсутсвуют пропуски

In [539]:
## Размерность датафрема comments
comments.shape

(90902,)

In [540]:
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: target
Non-Null Count  Dtype
--------------  -----
90902 non-null  int64
dtypes: int64(1)
memory usage: 710.3 KB


В датасете отсутсвуют пропуски

In [541]:
target.shape

(90902,)

### Задание 1 Теперь разделим наши данные на train и test. Пусть в тест у нас пойдет 30% данных. Для этого можете использовать библиотеку train_test_split из sklearn.

In [542]:
# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    comments, target,           # данные
    test_size=0.3,  # 30% — тестовая выборка, 70% — обучающая
    random_state=42 # фиксируем случайность для повторяемости
)

# Выводим результат
print("Тренировочные признаки:", X_train)
print("---------------------------------------")
print("Тренировочные метки:", y_train)
print("---------------------------------------")


Тренировочные признаки: 12294                                    Muslim terrorist.
57506    It's ironic that these are the same groups tha...
56118    Star Wars has a wow factor that Star Trek does...
28624    The settlement is 100% appropriate.\nEnding th...
63482    Where did it say to cover your cough with your...
                               ...                        
6265     You fit perfectly in Clinton's libdem basket o...
54886    I'll bet those independent contractors have no...
76820    "Lower tier" people, especially the young, wer...
860      The Devil made her do it and the man too becau...
15795    A leak in an 8" pipe, while obviously not a go...
Name: comment_text, Length: 63631, dtype: object
---------------------------------------
Тренировочные метки: 12294    1
57506    0
56118    0
28624    1
63482    0
        ..
6265     1
54886    0
76820    0
860      1
15795    1
Name: target, Length: 63631, dtype: int64
---------------------------------------


In [543]:
print("---------------------------------------")
print("Тестовые признаки:", X_test)
print("---------------------------------------")
print("Тестовые метки:", y_test)

---------------------------------------
Тестовые признаки: 80470    Not meaning to belittle your practical concern...
28773    Did you mean to imply that the bears are dying...
76685    As a 9-year dispatch veteran, let me break it ...
12580    A your wimp is traveling across Canada instead...
16111    Oh Dispatch, you right wing media elitist rag....
                               ...                        
27748    The problem with Trump's misogyny is that when...
86400                                                 Why?
40444    Bull crap!!! ask the Venezuelan's if their \n"...
53979    Well if you read and understood my posts, you ...
13216    you guys really still think that after 40 year...
Name: comment_text, Length: 27271, dtype: object
---------------------------------------
Тестовые метки: 80470    0
28773    1
76685    0
12580    1
16111    1
        ..
27748    1
86400    0
40444    1
53979    0
13216    1
Name: target, Length: 27271, dtype: int64


### Задание 2 Преобразуйте текст, который вы поделили на train и test, в числовой формат с помощью функции CountVectorizer.

In [544]:
# Инициализация векторизатора
vectorizer = CountVectorizer(max_features=200)

# Преобразование датафрема X_train в новый датафрейм, в котором тексты заменены на вектора  
X_train_vec = vectorizer.fit_transform(X_train)


# Преобразование датафрема X_train в новый датафрейм, в котором тексты заменены на вектора  
X_test_vec = vectorizer.fit_transform(X_test)

## Сверяем размерность полученных датасетов 
print(f'Размерноть датасета X_train_vec', X_train_vec.shape)
print(f'Размерность датасета y_train', y_train.shape)

print('------------------------------------------------------')
print(f'Размерноть датасета X_train_vec', X_test_vec.shape)
print(f'Размерность датасета y_train', y_test.shape)


Размерноть датасета X_train_vec (63631, 200)
Размерность датасета y_train (63631,)
------------------------------------------------------
Размерноть датасета X_train_vec (27271, 200)
Размерность датасета y_train (27271,)


#### Задание 3 Теперь в качестве модели, которая будет классифицировать нам комментарии на токсичные и нетоксичные, возьмем логистическую регрессию. Импортируйте из библиотеки sklearn логистическую регрессию LogisticRegression с параметром max_iter=2000. Для оценки модели возьмите метрику accuracy и посчитайте ее

In [545]:
# Инициализация и обучение модели логистической регрессии
model = LogisticRegression(max_iter=2000, warm_start=True)
model.fit(X_train_vec, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test_vec)

# Расчет accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Метрика Accuracy: {accuracy:.4f}')

Метрика Accuracy: 0.6065


Получена метрика Accuracy - 0.6065

### Задание 4 Чтобы мы смогли протестировать разные комментарии, которые приходят в голову, пропишите ниже функцию, для которой на вход мы бы подавали наш комментарий, а на выход получали предсказание, насколько от 0 до 1 комментарий является токсичным.

In [546]:
def check_toxic(text):
	
	if model.predict(vectorizer.transform([text])):
		return print('Токсичное высказываение')
	else:
		return print ('Нормальное высказываение')

# Проверка

test_text1 = "Apples are stupid"
test_text2 = "I love apples"

check_toxic(test_text1)
check_toxic(test_text2)



Токсичное высказываение
Нормальное высказываение


Подтверждаем, что молель корректно класифицирует высказывания 

#### Задание 6 Если ваш алгоритм работает корректно, то комментарий «I love apples» должен быть определен как нетоксичный, а «Apples are stupid» — как токсичный. 

#### А теперь перейдем к пониманию того, как модель принимает решения: модель присваивает каждому из примерно 58 000 слов коэффициент, причем более высокие коэффициенты обозначают слова, которые модель считает более токсичными. 

#### Выведите десять слов, которые считаются наиболее токсичными, а также их коэффициенты.
#### Hint: в этом вам поможет атрибут vectorizer.vocabulary_.keys() и classifier.coef_

In [553]:
## формируем словарь и коэффиценты
vocabls = vectorizer.vocabulary_
coefs = model.coef_[0]

index_to_word = {index: word for word, index in vocabls.items()}

# Получаем 10 первых токсичных слов
top_indices = np.argsort(coefs)[-10:][::-1]
top_toxic_words = [(index_to_word[i], coefs[i]) for i in top_indices]

# Вывод
print("10 токсичных слов:")
print("-------------------------")
for word, weight in top_toxic_words:
    print(f"{word}: {weight}")

10 токсичных слов:
-------------------------
idiot: 5.732400434893729
idiots: 5.659027706355476
such: 5.510022423357099
ignorant: 3.7407246727464765
who: 0.9464284920846445
man: 0.5627805343047959
they: 0.41925204665651905
two: 0.3894660106423257
left: 0.3184341495570787
work: 0.2971392572980749


####  Задание 7 Взгляните на самые токсичные слова из задания 6. 

#### Вызывают ли у вас удивление какие-нибудь из них? 

Можно отметить, что большая часть слов из представленных в списке не явлюятся токсичными и не носят неготивной окрски

#### Есть ли слова, которых, кажется, не должно быть в списке?

Очевидно, что слова such (такой), who (кто), man (мужчина), they (они), ецщ (два), left (лево), work (работа)
не могут быть отнесены к токсичным 

### Задание 8 Давайте попробуем протестировать модель на ее предвзятость, например, к религии.

In [ ]:
check_toxic("I have a christian friend")
check_toxic("I have a muslim friend")
check_toxic("I have a white friend")
check_toxic("I have a black friend")

## Все высказывания отнесены к нормльным высказываениям. Можно сделать вывод о том, что у модели нет предвзятости.

Нормальное высказываение
Нормальное высказываение
Нормальное высказываение
Нормальное высказываение


#### Задание 9 Немного теории: 
#### в этике ИИ принято писать такой алгоритм, который будет соответствовать 4 критериям этики:

Предвзятость может приветси к тому, что даже обычное упоминение слов коэффицент токсичности которых выше, может повысить токсичность высказывания в целом. Может приводить к дескриминации по религиозному признаку.



## Задание 10 Подумайте о том, как можно улучшить алгоритм, чтобы сделать его более этичным. Напишите 1–2 идеи.

1. Возможно имело бы смысл проводить векторизацию текстов не на отдельные слова, а на словосочетания. 
Представляется, что такой подход повысит точность класификации текстов

2. Для тренировки алгримтмов размечать датасеты не на 2 категории, а на 3 (токсичное, нейтральное, положительное)

